In [2]:
import pandas as pd
import numpy as np
import math
from scipy import stats

# Q1

In [68]:
df = pd.read_csv('AB_test_data.csv')

In [69]:
df

,purchase_TF,Variant,date,id
0,False,A,2019-11-08,0x25b44a
1,False,B,2020-08-27,0x46271e
2,False,A,2020-06-11,0x80b8f1
3,False,B,2020-08-22,0x8d736d
4,False,A,2020-08-05,0x96c9c8
...,...,...,...,...
129995,False,A,2020-07-23,0x4089c2
129996,False,A,2020-06-24,0x6a5e3a
129997,False,A,2019-10-12,0x95e302
129998,False,A,2020-03-18,0x7c4afa


In [71]:
df['purchase_TF'] = df['purchase_TF'].astype(str)

In [72]:
df['purchase_TF'] = df['purchase_TF'].map(
                   {"True":1 ,"False":0})

In [73]:
df

,purchase_TF,Variant,date,id
0,0,A,2019-11-08,0x25b44a
1,0,B,2020-08-27,0x46271e
2,0,A,2020-06-11,0x80b8f1
3,0,B,2020-08-22,0x8d736d
4,0,A,2020-08-05,0x96c9c8
...,...,...,...,...
129995,0,A,2020-07-23,0x4089c2
129996,0,A,2020-06-24,0x6a5e3a
129997,0,A,2019-10-12,0x95e302
129998,0,A,2020-03-18,0x7c4afa


In [ ]:
# H0: P_A = P_B
# H1: P_A < P_B

In [84]:
res = stats.ttest_ind(df[df['Variant']== 'A'].purchase_TF, 
                      df[df['Variant']== 'B'].purchase_TF,
                      equal_var='unequal', alternative='less')
print(f'p-value for single sided test: {res.pvalue:.10f}')

p-value for single sided test: 0.0000000844


In [ ]:
#Since p-value is very small, we reject the null hypothesis. 
#Therefore，alternative B improved conversion rates (site users book the property) over alternative A.

In [106]:
df_A = df[df.Variant == 'A']

In [107]:
df_A_T = df_A[df_A.purchase_TF == True]
df_A_F = df_A[df_A.purchase_TF == False]

In [108]:
df_A_T

,purchase_TF,Variant,date,id
10,1,A,2019-08-15,0x7c0fef
19,1,A,2020-05-09,0x676bfc
22,1,A,2020-05-26,0x7d6cc8
37,1,A,2020-06-10,0x57faca
45,1,A,2020-02-01,0x67527f
...,...,...,...,...
129978,1,A,2019-10-29,0x3f5ff0
129980,1,A,2020-04-17,0x893f70
129982,1,A,2019-11-04,0x891175
129988,1,A,2020-02-10,0x964ba6


In [109]:
#Conversion rate of A 
Con_A = (df_A_T.shape[0])/df_A.shape[0]

In [110]:
Con_A

0.149616

In [111]:
df_B = df[df.Variant == 'B']

In [112]:
df_B 

,purchase_TF,Variant,date,id
1,0,B,2020-08-27,0x46271e
3,0,B,2020-08-22,0x8d736d
59,0,B,2020-08-19,0x3ff83f
74,0,B,2020-08-10,0x138d19
103,0,B,2020-08-04,0x966e6a
...,...,...,...,...
129805,0,B,2020-08-27,0x7d95d0
129827,0,B,2020-08-10,0x38a6e8
129879,0,B,2020-08-30,0x6a711a
129910,0,B,2020-08-13,0x13175a


In [113]:
df_B_T = df_B[df_B.purchase_TF == True]

In [114]:
#Conversion rate of B 
Con_B = (df_B_T.shape[0])/df_B.shape[0]

In [115]:
Con_B

0.1766

In [116]:
#Since Con_B > Con_A, alternative B improved conversion rates over alternative A.

# Q2

Calculate the optimal sample size 

In [ ]:
#95% confidence rate and test with 80% power
# False positive type I error prob.=α

In [100]:
#optimal sample size
alpha = 0.05
beta = 1 - 0.8
MDE = Con_B-Con_A
Con_avg = (Con_A+Con_B)/2

In [101]:
import scipy.stats as stats
z_alpha = abs(stats.norm.ppf(q = 0.05))  
z_beta = abs(stats.norm.ppf(q = 0.8))  

In [94]:
z_alpha
z_beta

0.8416212335729143

In [102]:
op_sample = ((z_alpha*math.sqrt(2*Con_avg*(1-Con_avg))+z_beta*math.sqrt(Con_A*(1-Con_A)+Con_B*(1-Con_B)))**2)*(1/(MDE**2))

In [103]:
op_sample

2317.0427404559846

Conduct test 10 times

In [ ]:
# H0: P_A = P_B
# H1: P_A < P_B

In [121]:
for i in range(10):
    sample_A = df_A.sample(n = math.ceil(op_sample), random_state = i)
    sample_B = df_B.sample(n = math.ceil(op_sample), random_state = i)
    res = stats.ttest_ind(sample_A[sample_A['Variant']== 'A'].purchase_TF, 
                      sample_B[sample_B['Variant']== 'B'].purchase_TF,
                      equal_var='unequal', alternative='less')
    print("The p-value for test_%d is %f:" %(i, res.pvalue))

The p-value for test_0 is 0.017901:
The p-value for test_1 is 0.005302:
The p-value for test_2 is 0.200816:
The p-value for test_3 is 0.001849:
The p-value for test_4 is 0.000996:
The p-value for test_5 is 0.010904:
The p-value for test_6 is 0.083309:
The p-value for test_7 is 0.000780:
The p-value for test_8 is 0.015543:
The p-value for test_9 is 0.006150:


In [ ]:
# Since the confidence level is 95%, when p-value < 0.05, then we can reject the null hypothesis in 95% confidence level.
# Therefore, there are 2 hypothesis tests which cannot reject null hypothsis. 
# The rest of the tests reject the null hypothesis.

# Q3

In [163]:
n_list = []
for i in range(10):
    alpha = 0.05
    beta = 0.2
    ln_A = math.log(1/alpha)
    ln_B = math.log(beta)
    #print(ln_B)
    sample_A = df_A.sample(n = math.ceil(op_sample), random_state = i)
    sample_B = df_B.sample(n = math.ceil(op_sample), random_state = i)
    sample_A = sample_A.reset_index(drop = True)
    sample_B = sample_B.reset_index(drop = True)
    Con_A = sum(sample_A['purchase_TF'])/sample_A.shape[0]
    Con_B = sum(sample_B['purchase_TF'])/sample_B.shape[0]
    #print(Con_A)
    ln_lambda_n = 0
    for n in range(math.ceil(op_sample)):
        if sample_B.iloc[n]["purchase_TF"] == 1:
            ln_lambda_a = math.log(Con_B/Con_A)
            #print(ln_lambda_n)
        else:
            ln_lambda_a = math.log((1-Con_B)/(1-Con_A))
            
        ln_lambda_n = ln_lambda_n + ln_lambda_a
        
        if ln_lambda_n <= ln_B:
            n_list.append(n)
            print(str(n) + " " + "Accept H0")
            break
        
        if ln_lambda_n >= ln_A:
            n_list.append(n)
            print(str(n) + " " + "Reject H0")
            break
        
        if (n == (math.ceil(op_sample-1))  and (ln_lambda_n < ln_A and ln_lambda_n > ln_B)):
            print("N/A")
        
print(n_list)

344 Reject H0
502 Reject H0
N/A
1494 Reject H0
1275 Reject H0
564 Reject H0
N/A
273 Accept H0
268 Accept H0
532 Reject H0
[344, 502, 1494, 1275, 564, 273, 268, 532]


In [164]:
avg_iterations = sum(n_list)/len(n_list)

In [165]:
avg_iterations

656.5